# LLM agents with memory in neo4j


## Prepare the API credential

In [1]:
from autogen import ConversableAgent
from autogen import config_list_from_json

with open('../openai.credential', 'r') as file:
    key = file.read()
MODEL = 'gpt-4o-mini'

config_list = config_list_from_json(env_or_file="../CONFIG_LIST", filter_dict={"model": MODEL})
config_list[0]["api_key"] = key

### A cypher executor

In [2]:
from tools.CypherExecutor import CypherCodeExecutor

cypher_executor = ConversableAgent(
    "cypher_executor",
    human_input_mode= "NEVER",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": CypherCodeExecutor()})

### A graph expert

In [3]:
#TODO: should there something on how to use the schema?
graph_expert = ConversableAgent(
    "graph_expert",
    system_message = """You are curious about person, and you can use a tool (linked to a graph database) to store the information about the persons you heard. Once you get a description about a person, find the person in the graph database as a node (or create one if it does not exist in the database), and save the description as well. After that, please identify all the other persons mentioned in this description, save them into the graph as nodes, together with the relation between the original person and the other person. Finally, find one person from the graph who does not have a description yet, and say 'I want to know more about <the name of this person>. TERMINATE'. Try not to query the database too often and to do so use cypher format""",
    llm_config = {"config_list": config_list},
    code_execution_config=False
)

### A schema dumper

In [4]:
from autogen import GroupChat, GroupChatManager
from tools.tools import retrieve_content, _reset_agents

#TODO: is there a way to have this as a generic function? in tools.tools
def state_transition(last_speaker, groupchat):
    if "cypher" in groupchat.messages[-1]["content"]: return cypher_executor
    elif last_speaker is person_expert: return graph_expert
    else: return person_expert

ret = retrieve_content()

person_expert = ConversableAgent(
    "person_expert",
    system_message = "You are a helpful assistant. Once you were asked about a person, reply a short description (<100 words) about this person, including his/her relation with some other persons in the description.",
    llm_config = {"config_list": config_list},
    #is_termination_msg=lambda msg: "" in msg["content"].lower(),
    human_input_mode="NEVER",
)

groupchat = GroupChat(agents=[person_expert, graph_expert, cypher_executor, ret], messages=[],
                          max_round=10, speaker_selection_method=state_transition)
    
manager   = GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list})

PROBLEM = "I want to know something about Zhou Shuren"

_reset_agents(groupchat)

In [5]:
ret.initiate_chat(
    manager,
    message=ret.message_generator,
    problem=PROBLEM,
    max_turns=2)

Trying to create collection.


/opt/conda/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
Number of requested results 20 is greater than number of elements in index 1, updating n_results = 1


doc_ids:  [['doc_0']]
Adding content of doc doc_0 to context.
retriever (to chat_manager):

DATABASE SCHEMA
[
    {
        "nodes": [],
        "relationships": []
    }
]


TASK
I want to know something about Zhou Shuren

--------------------------------------------------------------------------------

Next speaker: person_expert

[autogen.oai.client: 10-25 08:43:16] {315} WARNING - Model gpt-4o-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
person_expert (to chat_manager):

Zhou Shuren, better known by his pen name Lu Xun, was a prominent Chinese writer and thinker in the early 20th century. He is celebrated for his critical essays and short stories that explored social issues and the human condition. Lu Xun was also a key figure in the May Fourth Movement, advocating for cultural reform. He was deeply influenced by fellow intellectuals such as Chen Duxiu, and he often com

[]

cypher_executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 

--------------------------------------------------------------------------------

Next speaker: person_expert

[autogen.oai.client: 10-25 08:43:29] {315} WARNING - Model gpt-4o-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
person_expert (to chat_manager):

Chen Duxiu was a significant Chinese revolutionary and intellectual leader during the early 20th century, known for his role in the May Fourth Movement. He co-founded the Chinese Communist Party and was a prominent advocate for modernization and democracy in China. Chen was also a key figure in promoting new cultural and literary movements, often emphasizing the need for social reform. His interactions and ideological debates with contemporaries like Lu Xun profoundly shaped modern Chinese thought and literature.

-------------------------

[]

cypher_executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 

--------------------------------------------------------------------------------

Next speaker: person_expert

[autogen.oai.client: 10-25 08:43:37] {315} WARNING - Model gpt-4o-mini is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
person_expert (to chat_manager):

Lu Xun, born Zhou Shuren, was an influential Chinese writer and thinker known for his sharp critiques of society and exploration of human psychology. Active during the early 20th century, he challenged traditional norms through his essays and short stories, addressing issues of social injustice and cultural stagnation. Lu Xun was a leading figure in the May Fourth Movement, advocating for national rejuvenation and literary reform. His literary legacy continues to inspire and provoke thought, making him one of the most significant figures i

[]

cypher_executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'DATABASE SCHEMA\n[\n    {\n        "nodes": [],\n        "relationships": []\n    }\n]\n\n\nTASK\nI want to know something about Zhou Shuren', 'role': 'assistant'}, {'content': 'Zhou Shuren, better known by his pen name Lu Xun, was a prominent Chinese writer and thinker in the early 20th century. He is celebrated for his critical essays and short stories that explored social issues and the human condition. Lu Xun was also a key figure in the May Fourth Movement, advocating for cultural reform. He was deeply influenced by fellow intellectuals such as Chen Duxiu, and he often commented on the tumultuous relationships within Chinese society during his time. His works remain influential in modern Chinese literature.', 'name': 'person_expert', 'role': 'user'}, {'content': '```cypher\nMERGE (zhoushuren:Person {name: "Zhou Shuren", pen_name: "Lu Xun"})\nSET zhoushuren.description = "Zhou Shuren, better known by his pen name Lu Xun, was a pro